# __Diffusion Tensor Imaging (DTI) analysis notebook__
#### __Last updated on:__ 19/11/2019
#### __Author:__ Rakshit Dadarwal

### __Requirements:__
#### 1. Single or multi-shell DWI data
#### 2. DIPY (https://dipy.org/)

### __This script includes:__
#### 1. DTI parametric maps estimation
#### 2. Compute color-coded or RGB Fractional Anisotropy (FA) map 

In [3]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
%matplotlib inline


#--------------------------------------------------------------
#        Load pre-processed DWI data and brain mask 
#--------------------------------------------------------------
data_path = "/home/User/Diffusion"
file1 = 'eddy_corrected.nii.gz'
path1 = os.path.join(data_path,file1)

if os.path.exists(path1):
        
    print('Reading pre-processed diffusion data')
    img1 = nib.load(path1)
    data = img1.get_data()

    file2 = 'brain_mask.nii.gz'
    path2 = os.path.join(data_path, file2)
    img2 = nib.load(path2)
    mask = img2.get_data()

    fbval = data_path+'/dMRI.bvals'
    fbvec = data_path+'/eddy_corrected.eddy_rotated_bvecs'
    from dipy.io import read_bvals_bvecs
    bvals, bvecs = read_bvals_bvecs(fbval,fbvec)
        
    from dipy.core.gradients import gradient_table
    gtab = gradient_table(bvals, bvecs)

    #--------------------------------------------------------------
    #               Fit diffusion tensor model
    #--------------------------------------------------------------
    print('Fitting diffuison tensor model')
    from dipy.reconst.dti import TensorModel
    ten_model = TensorModel(gtab)

    ten_fit = ten_model.fit(data, mask)
        
    #--------------------------------------------------------------
    #               Save DTI parametric maps
    #--------------------------------------------------------------
    if not os.path.exists(data_path+'/DTI/'):
        os.mkdir(data_path+'/DTI')
    output_path = data_path+'/DTI/'
        
    FA = ten_fit.fa
    AD = ten_fit.ad
    RD = ten_fit.rd
    MD = ten_fit.md
        
    nib.save(nib.Nifti1Image(FA, img1.affine), os.path.join(output_path,'FA.nii.gz'))
    nib.save(nib.Nifti1Image(MD, img1.affine), os.path.join(output_path,'MD.nii.gz'))
    nib.save(nib.Nifti1Image(RD, img1.affine), os.path.join(output_path,'RD.nii.gz'))
    nib.save(nib.Nifti1Image(AD, img1.affine), os.path.join(output_path,'AD.nii.gz'))
    
    #--------------------------------------------------------------
    #               Save FA RGB map
    #--------------------------------------------------------------
    from dipy.reconst.dti import fractional_anisotropy
    fa = fractional_anisotropy(ten_fit.evals)
    from dipy.reconst.dti import color_fa
    cfa = color_fa(fa, ten_fit.evecs)
    FA = np.clip(fa, 0, 1)
    RGB = color_fa(fa, ten_fit.evecs)

    nib.save(nib.Nifti1Image(np.array(255 * cfa, 'uint8'), img1.affine), os.path.join(output_path,'FA_RGB.nii.gz'))

    print('Processing completed')
    print('Enjoy!!')

Reading pre-processed diffusion data
Fitting diffuison tensor model
Processing completed
Enjoy!!
